In [7]:
pip install pyvis beautifulsoup4


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
from bs4 import BeautifulSoup
import urllib.parse


def flatten(xss):
    return [x for xs in xss for x in xs]

def crawl_everything_for_a_card_name(card_name, color_name):
    card_name_url = urllib.parse.quote_plus(card_name)
    url ='https://commanderspellbook.com/search/?q='+card_name_url+'+ci%3A'+color_name+'+legal%3Acommander'
    #print(url)
    combos = crawl_until_the_end(url)
    return combos

def crawl_until_the_end(url):
    current_page = 1
    combos = []
    combos_found = True

    while combos_found:
        url_next = url + '&page=' + str(current_page )
        response = requests.get(url_next)

        soup = BeautifulSoup(response.content, 'html.parser')
        s = soup.find_all('h3', class_="heading-title", string='No Combos Found')
        if s:
            combos_found=False
        combos.append(crawl(response))
        current_page += 1
    return flatten(combos)

def crawl(response):
    if response.status_code == 200:
        combos = []
        soup = BeautifulSoup(response.content, 'html.parser')
        combo_results = soup.find_all('a', class_='comboResults_comboResult__VcfMx')
        for combo in combo_results:
            cards_html = combo.find_all('div', class_="card-name")
            images_html = combo.find_all('div', class_='cardTooltip_cardTooltip__3eItj')
            results_html = combo.find_all('div', class_="result")

            cards = []
            images = []
            results = []

            for i, card in enumerate(cards_html):
                cards.append(card.text.strip())
            for i, image in enumerate(images_html):
                src = image.find('img')['src']
                images.append(src)
            for i, result in enumerate(results_html):
                results.append(result.text.strip())

            combo = {
                "cards": cards,
                "images": images,
                "results": results
            }
            print(combo)

            combos.append(combo)
        return combos
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

def digging_deeper(card_name, color_name):
    combos = crawl_everything_for_a_card_name(card_name, color_name)

    combo_pieces = []
    for combo in combos:
        cards = combo.get("cards")
        combo_pieces.append(cards)
    combo_pieces = set(flatten(combo_pieces))
    combo_pieces.remove(card_name)
    print(combo_pieces)

    combos_plus_one = []
    combos_plus_one.append(combos)
    for combo_piece in combo_pieces:
        c = crawl_everything_for_a_card_name(combo_piece, color_name)
        combos_plus_one.append(c)

    combos_plus_one = flatten(combos_plus_one)
    print(combos_plus_one)
    return combos_plus_one


In [9]:
from cProfile import label
from pyvis.network import Network
import random

def add_all_edges(net, cards, results):
    r = lambda :random.randint(0, 255)
    color  ='#%02X%02X%02X' % (r(), r(), r())
    for card in cards:
        for other_card in cards:
            if card == other_card:
                continue
            net.add_edge(card, other_card, color=color, title='\n'.join(results))

def create_graph(combos):
    net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, select_menu=True,
                  filter_menu=True)
    net.barnes_hut()
    if not combos:
        return
    for combo in combos:
        cards = combo.get("cards")
        images = combo.get("images")
        for i, card in enumerate(cards):
            net.add_node(card, title=card, shape="image", image=images[i], size=20, mass=1)
        add_all_edges(net, cards, combo.get("results"))

    # set the size in relation to the number of connections
    for node in net.get_nodes():
        neighbors = net.neighbors(node)
        node_id = net.get_node(node)
        node_id['size'] = min(210, 20 + 10 * len(neighbors))
        node_id['mass'] = min(100, 1 + len(neighbors))

    net.show_buttons(filter_="physics")
    net.toggle_physics(True)
    net.show("cards.html")

cards_mock = ['Xyris, the Writhing Storm', 'Intruder Alarm', 'Lore Broker']
images_mock = ['https://api.scryfall.com/cards/named?format=image&version=normal&exact=Xyris%2C%20the%20Writhing%20Storm&face=front', 'https://api.scryfall.com/cards/named?format=image&version=normal&exact=Intruder%20Alarm&face=front', 'https://api.scryfall.com/cards/named?format=image&version=normal&exact=Lore%20Broker&face=front']
results_mock = ['Infinite draw triggers for all players', 'Infinite looting for all players',
                    'Infinite self-discard triggers for all players', 'Near-infinite creature tokens',
                    'Near-infinite ETB', 'Near-infinite untap of all creatures']

t = {
    "cards": cards_mock,
    "images": images_mock,
    "results": results_mock
}

create_graph([t])


cards.html


In [10]:
combos_plus_one = []

# You want to add your colour identity and the cards you want to start crawling the spellbook with below:

In [11]:
color_name = "Azorius"
card_names = [
    "Abdel Adrian, Gorion's Ward",
    "Fiend Hunter",
    "Deadeye Navigator",
    "Phantom Steed",
    "Lumbering Battlement",
    "Suspension Field",
    "Glorious Protector",
    "Icewind Stalwart",
    "Wormfang Drake",
    "Teleportation Circle"
            ]

- `crawl_everything_for_a_card_name` looks for every combo in your colour identity that a given card is a part of
- `digging_deeper` runs `crawl_everything_for_a_card_name`, but then also looks for every combo with your newly found combo piece

In [12]:


for card_name in card_names:
    print(card_name)
    
    combos = crawl_everything_for_a_card_name(card_name, color_name)
    #combos = digging_deeper(card_name, color_name)

    combos_plus_one.append(combos)



Abdel Adrian, Gorion's Ward
{'cards': ["Abdel Adrian, Gorion's Ward", 'Restoration Angel', 'Ephemerate'], 'images': ["https://api.scryfall.com/cards/named?format=image&version=normal&exact=Abdel%20Adrian%2C%20Gorion's%20Ward&face=front", 'https://api.scryfall.com/cards/named?format=image&version=normal&exact=Restoration%20Angel&face=front', 'https://api.scryfall.com/cards/named?format=image&version=normal&exact=Ephemerate&face=front'], 'results': ['Infinite creature tokens', 'Infinite ETB', 'Infinite LTB']}
{'cards': ["Abdel Adrian, Gorion's Ward", 'Felidar Guardian', 'Ephemerate'], 'images': ["https://api.scryfall.com/cards/named?format=image&version=normal&exact=Abdel%20Adrian%2C%20Gorion's%20Ward&face=front", 'https://api.scryfall.com/cards/named?format=image&version=normal&exact=Felidar%20Guardian&face=front', 'https://api.scryfall.com/cards/named?format=image&version=normal&exact=Ephemerate&face=front'], 'results': ['Infinite creature tokens', 'Infinite ETB', 'Infinite LTB']}
{'ca

In [13]:
combos_plus_one = flatten(combos_plus_one)

In [14]:
removelist = [
    "Ashnod's Altar" ,
    'Defiler of Faith'
             , "Myr Battlesphere"
             , "Karmic Guide"
             , "Mycosynth Lattice"
             , "Sram's Expertise"
             , "Emry, Lurker of the Loch"
             , "Chromatic Orrery"
             , "Ornithopter"
             , "Encroaching Mycosynth"
             , "Palinchron"
             , "Glint Hawk"
             , "Whitemane Lion"
             , "Semblance Anvil"
             , "Deathrender"
             , "Altar of Dementia"
             , "Eldrazi Displacer"
             , "Myr Retriever"
             , "Phyrexian Walker"
             , "Phyrexian Altar"
             , "Dress Down"
             , "Mycosynth Golem"
             , "Memnite"
             , "Mind Over Matter"
             , "Dross Scorpion"
             , "Salvagin Station"
             , "Animation Module"
             , "Blasting Station"
             , "Nim Deathmantle"
             , "Altar of the Brood"
             , "Time Warp"
             , "Time Stretch"
             , "Walk the Aeons"
             , "Cloudstone Curio"
             , "Temporal Manipulation"
             , "Capture of Jingzhou"
             , "Sakashima of a Thousand Faces"
             , "Mysterious Limousine"
            ]

combo_to_be_removed = []

for combo in combos_plus_one:
    for b in removelist:
        if b in combo['cards']:
            print(combo['cards'])
            combo_to_be_removed.append(combo)

for c in combo_to_be_removed:
    if c in combos_plus_one:
        combos_plus_one.remove(c)

["Abdel Adrian, Gorion's Ward", 'Eldrazi Displacer']
["Abdel Adrian, Gorion's Ward", 'Karmic Guide', "Ashnod's Altar"]
["Abdel Adrian, Gorion's Ward", 'Karmic Guide', "Ashnod's Altar"]
["Abdel Adrian, Gorion's Ward", 'Karmic Guide', 'Phyrexian Altar']
["Abdel Adrian, Gorion's Ward", 'Karmic Guide', 'Phyrexian Altar']
["Abdel Adrian, Gorion's Ward", 'Karmic Guide', 'Blasting Station']
["Abdel Adrian, Gorion's Ward", 'Karmic Guide', 'Blasting Station']
["Abdel Adrian, Gorion's Ward", 'Nim Deathmantle', "Ashnod's Altar"]
["Abdel Adrian, Gorion's Ward", 'Nim Deathmantle', "Ashnod's Altar"]
["Abdel Adrian, Gorion's Ward", 'Sakashima of a Thousand Faces', 'Glasspool Mimic // Glasspool Shore']
["Abdel Adrian, Gorion's Ward", 'Sakashima of a Thousand Faces', 'Spark Double']
["Abdel Adrian, Gorion's Ward", 'Glorious Protector', 'Mysterious Limousine']
["Abdel Adrian, Gorion's Ward", 'Nim Deathmantle', 'Phyrexian Altar']
["Abdel Adrian, Gorion's Ward", 'Nim Deathmantle', 'Phyrexian Altar']
["Abd

In [15]:
create_graph(combos_plus_one)

cards.html


In [16]:
all_cards = []
for combo in combos_plus_one:
    cards = combo['cards']
    all_cards.append(cards)

all_cards = set(flatten(all_cards))

for c in all_cards:
    print(c)

Hushwing Gryff
Leonin Squire
Anointed Procession
Tidal Barracuda
Starfield of Nyx
Pegasus Guardian // Rescue the Foal
Kaho, Minamo Historian
Great Whale
Parallax Wave
Soulherder
Archaeomancer
Sage of Hours
Workhorse
Lumbering Battlement
Phantom Steed
Cryptic Trilobite
Prosperous Pirates
Journey to Nowhere
Splash Portal
Peregrine Drake
Hushbringer
Glimmerpoint Stag
Touch the Spirit Realm
Detention Sphere
Pestermite
Vedalken Orrery
Experimental Aviator
Essence Flux
Cloudshift
Medomai the Ageless
Glorious Protector
Coveted Jewel
Leonin Relic-Warder
Preston, the Vanisher
Deepglow Skate
Norn's Choirmaster
Supply Runners
Charming Prince
Opalescence
Teferi, Who Slows the Sunset
Flickerwisp
Phantasmal Image
Icy Prison
Denry Klin, Editor in Chief
Abdel Adrian, Gorion's Ward
Lion's Eye Diamond
Arcane Signet
Clever Impersonator
Leyline of Anticipation
Torpor Orb
Wormfang Drake
Genesis Chamber
Brago, King Eternal
Exchange of Words
Auramancer
Conjurer's Closet
Nexus of Fate
Wanderwine Prophets
Azor